In [1]:
import pymongo
from pymongo import MongoClient
import json
import tweepy
import twitter
from pprint import pprint
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('config2.ini')

CONSUMER_KEY      = config['mytwitter']['api_key']
CONSUMER_SECRET   = config['mytwitter']['api_secrete']
OAUTH_TOKEN       = config['mytwitter']['access_token']
OATH_TOKEN_SECRET = config['mytwitter']['access_secrete']

mongod_connect = config['mymongo']['connection']

In [3]:
client = MongoClient(mongod_connect)
db = client.tweet_db # create a database named tweet_db
tweet_collection = db.tweet_collection_job #create a collection called tweet_collection
tweet_collection.create_index([("id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'id_1'

In [ ]:
stream_auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
stream_auth.set_access_token(OAUTH_TOKEN, OATH_TOKEN_SECRET)

strem_api = tweepy.API(stream_auth)

In [ ]:
track = ['National Security Job'] # define the keywords, tweets contain JMU

#locations = [-78.9326449,38.4150904,-78.8816972,38.4450731] #defin the location, in Harrisonburg, VA

In [ ]:
class MyStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        print (status.id_str)
        try:
            tweet_collection.insert_one(status._json)
        except:
            pass
  
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = strem_api.auth, listener=myStreamListener)
myStream.filter(track=track)

In [4]:
rest_auth = twitter.oauth.OAuth(OAUTH_TOKEN,OATH_TOKEN_SECRET,CONSUMER_KEY,CONSUMER_SECRET)
rest_api = twitter.Twitter(auth=rest_auth)

In [5]:
count = 100 #number of returned tweets, default and max is 100
#geocode = "38.4392897,-78.9412224,50mi"  # defin the location, in Harrisonburg, VA
q = "National Security Job"                               #define the keywords, tweets contain JMU

In [6]:
search_results = rest_api.search.tweets( count=count,q=q)
statuses = search_results["statuses"]
since_id_new = statuses[-1]['id']
for statuse in statuses:
    try:
        tweet_collection.insert_one(statuse)
        pprint(statuse['created_at'])# print the date of the collected tweets
    except:
        pass

'Tue Nov 05 20:05:44 +0000 2019'
'Tue Nov 05 20:05:27 +0000 2019'
'Tue Nov 05 20:03:35 +0000 2019'
'Tue Nov 05 19:57:15 +0000 2019'
'Tue Nov 05 19:52:06 +0000 2019'
'Tue Nov 05 19:42:32 +0000 2019'
'Tue Nov 05 19:34:23 +0000 2019'
'Tue Nov 05 19:22:49 +0000 2019'
'Tue Nov 05 19:18:07 +0000 2019'
'Tue Nov 05 18:45:42 +0000 2019'
'Tue Nov 05 18:44:22 +0000 2019'
'Tue Nov 05 18:40:47 +0000 2019'
'Tue Nov 05 18:37:14 +0000 2019'
'Tue Nov 05 18:25:59 +0000 2019'
'Tue Nov 05 18:12:01 +0000 2019'
'Tue Nov 05 18:08:06 +0000 2019'
'Tue Nov 05 18:04:52 +0000 2019'
'Tue Nov 05 17:56:52 +0000 2019'
'Tue Nov 05 17:50:01 +0000 2019'
'Tue Nov 05 17:44:50 +0000 2019'
'Tue Nov 05 17:34:10 +0000 2019'
'Tue Nov 05 17:32:43 +0000 2019'
'Tue Nov 05 17:30:42 +0000 2019'
'Tue Nov 05 17:30:19 +0000 2019'
'Tue Nov 05 17:16:59 +0000 2019'
'Tue Nov 05 17:12:22 +0000 2019'
'Tue Nov 05 17:11:50 +0000 2019'
'Tue Nov 05 17:10:05 +0000 2019'
'Tue Nov 05 17:03:26 +0000 2019'
'Tue Nov 05 17:00:31 +0000 2019'
'Tue Nov 0

In [ ]:
since_id_old = 0
while(since_id_new != since_id_old):
    since_id_old = since_id_new
    search_results = rest_api.search.tweets( count=count,q=q,
                        geocode=geocode, max_id= since_id_new)
    statuses = search_results["statuses"]
    since_id_new = statuses[-1]['id']
    for statuse in statuses:
        try:
            tweet_collection.insert_one(statuse)
            pprint(statuse['created_at']) # print the date of the collected tweets
        except:
            pass

In [7]:
print(tweet_collection.estimated_document_count())# number of tweets collected

user_cursor = tweet_collection.distinct("user.id")
print (len(user_cursor)) # number of unique Twitter users

638
617


In [8]:
tweet_collection.create_index([("text", pymongo.TEXT)], name='text_index', default_language='english') # create a text index

tweet_cursor = tweet_collection.find({"$text": {"$search": "National Security Job"}}) # return tweets contain football
for document in tweet_cursor:
    try:
        print ('----')
#         pprint (document) # use pprint to print the entire tweet document
   
        print ('name:', document["user"]["name"]) # user name
        print ('text:', document["text"])         # tweets
    except:
        print ("***error in encoding")
        pass

----
name: Rachel Kurzius
text: Re: second jobs for CMs: "There are jobs, and there are jobs," says Cheh, who has a second job as a teacher, which… https://t.co/WIYnWVvdB3
----
name: Lindsay Reinhard
text: RT @Greg_CHF: The #LNP talks about jobs but they forget farm jobs are jobs too. Farm jobs are sustainable. #auspol https://t.co/nPLGKev3tY
----
name: 🇿🇦CollenM
text: RT @CamagwiniBazi: Best sources to find a job: 

-Entry level jobs: Indeed, JunkMail
-Medium,Associate level jobs - Pnet
-High level jobs -…
----
name: A-Aron
text: RT @CamagwiniBazi: Best sources to find a job: 

-Entry level jobs: Indeed, JunkMail
-Medium,Associate level jobs - Pnet
-High level jobs -…
----
name: Laurielizabeth
text: RT @ChrisJZullo: Looking at jobs and deficits

Reagan: 15M jobs
$79B to $159B

Bush: 2M jobs
$153B to $255B

Clinton: 23M jobs
$255B to $12…
----
name: Jerry Bartels
text: RT @ChrisJZullo: Looking at jobs and deficits

Reagan: 15M jobs
$79B to $159B

Bush: 2M jobs
$153B to $255B

Clinton: 

text: RT @TrishQuenby: RCNi jobs fair at Leicester Tigers Stadium. Myself, Aimee and Lisa all proud to work for LPT @LPTnhs @evingtonmatron @sj_l…
----
name: Local Local Seattle
text: RT @JobsTimeless: Set/Lighting Designer at Cornerstone Technologies, LLC via (Timeless Jobs @jobstimeless) https://t.co/3eSciRBPKJ
----
name: TFJ‼️‼️
text: RT @CroSmith: Got ppl losing jobs over tweets, and losing their lives over chicken sandwiches. Dummies, too many followers not enough leade…
----
name: 👑
text: hillarious that people think my jobs glam but a full grown man projectile vomited over a seat that I had to sit on… https://t.co/qk4cv0L4SH
----
name: Melissa 🍋
text: Steve Jobs was in a foster home and you’re saying we should’ve been better off just aborting him instead of giving… https://t.co/1bVEgM1NbJ
----
name: RIX4TRUMP ⭐️⭐️⭐️
text: RT @Lrihendry: Donald Trump— A man with no political experience— did more for the US economy, American jobs, and world peace in 24 months t…
----
name: TMJ-MN 